In [11]:
from bs4 import BeautifulSoup
import requests
import numpy as np
import pandas as pd
from geopy.geocoders import Nominatim
import folium

In [12]:
url = 'https://en.wikipedia.org/wiki/List_of_neighbourhoods_in_Toronto'
result = requests.get(url)
print(url)
print(result.status_code)
print(result.headers)

https://en.wikipedia.org/wiki/List_of_neighbourhoods_in_Toronto
200
{'Date': 'Thu, 29 Jul 2021 07:50:10 GMT', 'Server': 'mw2408.codfw.wmnet', 'X-Content-Type-Options': 'nosniff', 'P3p': 'CP="See https://en.wikipedia.org/wiki/Special:CentralAutoLogin/P3P for more info."', 'Content-Language': 'en', 'Vary': 'Accept-Encoding,Cookie,Authorization', 'Last-Modified': 'Thu, 29 Jul 2021 00:34:00 GMT', 'Content-Type': 'text/html; charset=UTF-8', 'Content-Encoding': 'gzip', 'Age': '0', 'X-Cache': 'cp2033 miss, cp2039 miss', 'X-Cache-Status': 'miss', 'Server-Timing': 'cache;desc="miss", host;desc="cp2039"', 'Strict-Transport-Security': 'max-age=106384710; includeSubDomains; preload', 'Report-To': '{ "group": "wm_nel", "max_age": 86400, "endpoints": [{ "url": "https://intake-logging.wikimedia.org/v1/events?stream=w3c.reportingapi.network_error&schema_uri=/w3c/reportingapi/network_error/1.0.0" }] }', 'NEL': '{ "report_to": "wm_nel", "max_age": 86400, "failure_fraction": 0.05, "success_fraction": 0.0

In [13]:
df = pd.DataFrame(columns=['Hood', 'Latitude', 'Longitude'])
df.head()

,Hood,Latitude,Longitude


In [14]:
soup = BeautifulSoup(result.content, 'html.parser')
table = soup.find('table')
lis = table.find_all('li')

list_of_n = []
for li in lis:
    a = li.find('a')
    list_of_n.append(a.get('title').split(", ")[0].split(" (neighbourhood)")[0].split(" (Toronto)")[0] )

In [15]:
df['Hood'] = pd.Series(list_of_n)
print(df.shape)
df.head()

(32, 3)


,Hood,Latitude,Longitude
0,Alexandra Park,NaN,NaN
1,The Annex,NaN,NaN
2,Baldwin Village,NaN,NaN
3,Cabbagetown,NaN,NaN
4,CityPlace,NaN,NaN


In [16]:
df.drop_duplicates(inplace=True)
print(df.shape)
df.head()

(32, 3)


,Hood,Latitude,Longitude
0,Alexandra Park,NaN,NaN
1,The Annex,NaN,NaN
2,Baldwin Village,NaN,NaN
3,Cabbagetown,NaN,NaN
4,CityPlace,NaN,NaN


In [17]:
to_drop_unknown = []
geolocator = Nominatim(user_agent="coursera")
for index, row in df.iterrows():
    address = row['Hood'] + ', Toronto'
    try:
        location = geolocator.geocode(address)
        latitude = location.latitude
        longitude = location.longitude
        print('The geograpical coordinate of {} are {}, {}.'.format(address, latitude, longitude))
        df.loc[index, 'Latitude'] = latitude
        df.loc[index, 'Longitude'] = longitude
    except AttributeError:
        print('Cannot do: {}, will drop index: {}'.format(address, index))
        to_drop_unknown.append(index)

df.head()

The geograpical coordinate of Alexandra Park, Toronto are 43.650786999999994, -79.40431814731767.
The geograpical coordinate of The Annex, Toronto are 43.6703377, -79.407117.
The geograpical coordinate of Baldwin Village, Toronto are 43.6776885, -79.4081645.
The geograpical coordinate of Cabbagetown, Toronto are 43.6644734, -79.3669861.
The geograpical coordinate of CityPlace, Toronto are 43.6392482, -79.3963865.
The geograpical coordinate of Chinatown, Toronto are 43.6529237, -79.3980316.
The geograpical coordinate of Church and Wellesley, Toronto are 43.6655242, -79.3838011.
The geograpical coordinate of Corktown, Toronto are 43.6555594, -79.358487.
The geograpical coordinate of Discovery District, Toronto are 43.6575555, -79.3894803.
The geograpical coordinate of Distillery District, Toronto are 43.6502803, -79.3595767.
The geograpical coordinate of Toronto Entertainment District, Toronto are 43.65238435, -79.38356765.
Cannot do: East Bayfront, Toronto, will drop index: 11
The geogr

,Hood,Latitude,Longitude
0,Alexandra Park,43.6508,-79.4043
1,The Annex,43.6703,-79.4071
2,Baldwin Village,43.6777,-79.4082
3,Cabbagetown,43.6645,-79.367
4,CityPlace,43.6392,-79.3964


In [18]:
clean_df = df.drop(to_drop_unknown)

In [19]:
clean_df.shape

(31, 3)

In [20]:
address = 'Toronto'
try:
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    print('The geograpical coordinate of {} are {}, {}.'.format(address, latitude, longitude))
    df.loc[index, 'Latitude'] = latitude
    df.loc[index, 'Longitude'] = longitude
except AttributeError:
    print('Cannot do: {}, will drop index: {}'.format(address, index))

my_map = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(clean_df['Latitude'], clean_df['Longitude'], clean_df['Hood']):
    label = folium.Popup(label)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(my_map)  
    
my_map

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.
